# NAVER 한달살기 - topic modeling

 참조사이트
 - https://hyemin-kim.github.io/2020/08/29/E-Python-TextMining-2/

In [1]:
# !pip install matplotlib seaborn sklearn konlpy scikit-learn wordcloud pyLDAvis gensim

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re

from wordcloud import WordCloud, STOPWORDS

from konlpy.tag import Okt
okt = Okt() 
from collections import Counter

In [2]:
dong_list = '건입동 내도동 노형동 도남동 도두이동 도두일동 도련이동 도련일동 도평동 봉개동 삼도이동 삼도일동 삼양삼동 삼양이동 삼양일동 아라이동 아라일동 연동 영평동 오등동 오라삼동 오라이동 오라일동 외도이동 외도일동 용강동 용담삼동 용담이동 용담일동 월평동 이도이동 이도일동 이호이동 이호일동 일도이동 일도일동 해안동 화북이동 화북일동 회천동 구좌읍 애월읍 우도면 조천읍 추자면 한경면 한림읍'
dong_list = dong_list.split(" ")

for i in dong_list:
    if (i.find('읍')) > 0 or (i.find('면') > 0):
        print(i)

구좌읍
애월읍
우도면
조천읍
추자면
한경면
한림읍


In [3]:
df = pd.read_csv('./data/naver_month.csv')
df

,title,contents,time,blog_text,key
0,돌아기랑 제주도한달살기 24일째(406일) : 비양도,첨에 제주도 한달살기 올때도 어디갈지 생각을 안하고 와서 매번 전날이나 아침에 오늘...,2023.11.27,첨에 제주도 한달살기 올때도 어디갈지 생각을 안하고 와서 매번 전날이나 아침에 오늘...,한달살기
1,"제주도 한달살기 (20) - 우도 코코나라, 문빵구, 돌담집, 몬딱...",한달살기 중 두번째 우도 ㅎㅎ 처음은 전기차를 빌렸지만 이번엔 카트형식으로 나란히 ...,2023.11.07,한달살기 중 두번째 우도 ㅎㅎ처음은 전기차를 빌렸지만 이번엔 카트형식으로 나란히 앉...,한달살기
2,제주동쪽숙소 제주도한달살기하기 좋은 구좌 제주오션뷰2차♪,#제주오션뷰2차 #제주도펜션 #제주동쪽숙소 제주에는 크게 동쪽과 서쪽으로 나뉘면서 ...,2023.12.13,#제주오션뷰2차#제주도펜션#제주동쪽숙소제주에는 크게 동쪽과 서쪽으로 나뉘면서 다양한...,한달살기
3,제주도 한달살기 비용 (한달살이 생활비),제주도 한달살기는 돈이 적게 들지는 않지만 우리 가족에게 꽤 많은 추억을 남겼으므로...,2022.02.23,"2021년 4월에 제주도 한달살기를 하고 왔다.세식구 기준(어른 둘, 아기 하나)으...",한달살기
4,제주도 한달살기 숙소 비용 제주 중문 애견동반 더맨션 이이공공,그중에서도 제일 마음에 들었던 건 오후 시간 깊숙이 들어오는 가을 햇살 창밖에 보이...,2023.10.14,이번에는 좀 여유로운 여행을 하고 싶어서다소 긴 일정으로 제주도에 방문했어요.여행...,한달살기
...,...,...,...,...,...
1932,제주도한달살이,드뎌 육지다~ 뒤로 보이는 육지가 제주항이네요. 3다도인 제주에 좋은 추억도 많이 ...,2020.12.18,#제주도 #제주한달살이#실버크클라우드코로나19로 발이묶인 윈드서퍼들의차선택은~~ 한...,한달살이
1933,[제주도 한달살이] 제주도 게스트하우스 스텝 후기...,"연돈 예약 성공해서 연돈도 먹고, 한라산 등산도 가고, 스쿠버다이빙도 배우고, 저녁...",2023.05.28,22년12월28일~23년2월4일약 한달동안 제주살이를 했다.(글 마지막에 스텝 꿀팁...,한달살이
1934,제주도 한달살이 시작!!!,무계획으로 출발 하는 제주도 한달살이!!! 퇴사를 하고 나니 새로운 환경에서 살아보...,2022.03.16,무계획으로 출발 하는 제주도 한달살이!!!퇴사를 하고 나니 새로운 환경에서 살아보고...,한달살이
1935,[제주도 한달살이 이야기] 평산제면소에서 마제소바먹고 러닝후...,제주도에선 술 잘 안먹겠다고 다짐했는데 거의 하루 한캔은 아니여도 한모금씩은 매일 ...,2022.02.20,오늘은 숙소 근처에 있다는 마제소바 집에 들렸어요맛집이라고 해서 저번에 왔었는데그때...,한달살이


In [4]:
# df 수집 오류 결과 확인
(df == 'error').sum()

title        0
contents     0
time         0
blog_text    0
key          0
dtype: int64

In [5]:
df.isnull().sum()
# blog_text가 nan인 행 존재 -> 제외

title        0
contents     0
time         0
blog_text    3
key          0
dtype: int64

In [6]:
df[df['blog_text'].isna() == 1]

,title,contents,time,blog_text,key
1807,제주도 한달살이 스물한번째 날,제주도 한달살이 스물한번째 날입니다 오늘은 제주도에 오후부터 비가 예보 되있어서 나...,2022.03.30,NaN,한달살이
1882,제주도 한달살이 스물여덟번째 날,제주도 한달살이 스물여덟번째 날입니다 이제 제주도 살이 삼일 남았습니다 이번주는 날...,2022.04.07,NaN,한달살이
1919,제주도 한달살이 열다섯번째 날,제주도 한달살이 열다섯번째 날 입니다 벌써 한달중 절반이 지나가는군요 아침에 일어나...,2022.03.24,NaN,한달살이


In [7]:
# blog_text가 nan인 겨우 제외
df = df[df['blog_text'].isna() != 1]

### 토픽모델링
https://velog.io/@mare-solis/LDA-%ED%86%A0%ED%94%BD-%EB%AA%A8%EB%8D%B8%EB%A7%81%EC%9C%BC%EB%A1%9C-%EC%BD%98%ED%85%90%EC%B8%A0-%EB%A6%AC%EB%B7%B0%EB%A5%BC-%EB%B6%84%EC%84%9D%ED%95%98%EC%9E%90

In [376]:
# def text_cleaning(x):
#     nous = okt.nouns(hangul_function(x))
#     nous = [x for x in nous if len(x) > 1]  # 한글자 키워드 제거    
#     nous = [word for word in nous if not word in stopwords]
#     return nous

# temp_list = []
# for blog in df['blog_text']:    
#     temp_list.append(text_cleaning(blog))

In [8]:
def hangul_function(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

In [11]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric


import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [10]:
stopwords = set([])

web_stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
web_stopwords[:10]

for i in web_stopwords:
    stopwords.add(i[0])

print(len(stopwords))

jeju_stopwords = [ '리뷰', '숙소', '정말', '동안', '다음', '바로', '근처', '조금', '시간',
                   '호스트', '정도', '방문', '사장', '이용', '다시', '예약', '살이', '살기', '아주', '추천', '하루', '이번',
                   '처음', '달동', '저희', '하나', '사용', '모두', '진짜', '일주일', '보름', '선택', '시작', '한번', '가장', '마음',
                   '제외', '이동', '무엇', '준비', '그냥', '감사',  '덕분', '후기', '기회',  '다른', '여기', '우리',
                   '미스터', '미스터멘션', '멘션', '때문', '오늘', '도착', '보고', '마지막', '가지', '이제', '지금', '모습', '위해',
                   '타고', '사실', '포스팅', '주문', '역시', '침대', '거실', '매일', '출발', '소개', '기억', '참고', '계속', '객실',
                   '일차', '이야기', '요즘', '이유', '경우', '일단', '거의', '스텝', '제일', '자주', '어디', '마무리', '아래', '운영',
                   '이상', '의자', '별로', '원래', '중간', '뭔가', '첫째', '체크', '잠시', '언제', '살짝', '해도', '계단', '부분', '냉장고',
                   '침실', '기본', '제공', '일찍', '인테리어', '완전', '추가', '등등', '매우', '가야', '그때', '기간', '마치', '가득', '대부분',
                   '미리', '둘째', '근무', '공간', '스테이', '알파', '엄마', '아빠', '아들', '일정', '계획', '아침', '주방', '서울', '육지',
                   '점심', '저녁', '자리', '두리', '언니', '가격', '건물', '숙박', '만원', '가기', '코로나' ,                   
                   '호텔', '펜션', '게스트하우스', '독채','화장실', '욕실', '숙박', '세탁기', '가면', '직접', '비용', '길이',
                    '여행', '트립', '제주', '제주도', '생각', '느낌',
                    '광고', '스텝', '블로그', '개월', '도서관', '스탭'
                   ]
for word in jeju_stopwords:
    stopwords.add(word)

print(len(stopwords))

594
739


### 토픽 모델링 완성

In [12]:
temp_list = []
for blog in df['blog_text']:
    LDA_nous = okt.nouns(hangul_function(blog))
    LDA_nous = [x for x in LDA_nous if len(x) > 1]  # 한글자 키워드 제거    
    LDA_nous = [word for word in LDA_nous if not word in stopwords]
    temp_list.append(LDA_nous)

dictionary = corpora.Dictionary(temp_list)
dictionary.filter_extremes(no_below=2, no_above=0.5)
corpus_LDA = [dictionary.doc2bow(text) for text in temp_list]

num_topics = 5
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus_LDA,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

lda_visualization = gensimvis.prepare(model, corpus_LDA, dictionary, sort_topics=True)
pyLDAvis.save_html(lda_visualization, './result/topicmodeling_complete.html')

In [18]:
model.get_document_topics(corpus_LDA)

In [21]:
from pprint import pprint
pprint(model.print_topics())
doc_lda = model[corpus_LDA]

[(0,
  '0.015*"오름" + 0.015*"코스" + 0.012*"바다" + 0.011*"올레길" + 0.008*"해수욕장" + '
  '0.008*"사람" + 0.008*"해변" + 0.007*"풍경" + 0.006*"숲길" + 0.005*"한라산"'),
 (1,
  '0.009*"바다" + 0.007*"위치" + 0.007*"하우스" + 0.006*"가족" + 0.006*"서귀포" + '
  '0.005*"거리" + 0.004*"시설" + 0.004*"마당" + 0.003*"카페" + 0.003*"애월"'),
 (2,
  '0.018*"맛집" + 0.014*"아기" + 0.013*"카페" + 0.009*"식당" + 0.008*"돼지" + 0.007*"메뉴" '
  '+ 0.006*"고기" + 0.006*"바다" + 0.005*"애월" + 0.005*"음식"'),
 (3,
  '0.019*"카페" + 0.014*"사람" + 0.007*"바다" + 0.007*"친구" + 0.005*"날씨" + 0.004*"커피" '
  '+ 0.004*"비행기" + 0.004*"구경" + 0.004*"공항" + 0.004*"맛집"'),
 (4,
  '0.012*"체험" + 0.008*"바다" + 0.007*"우도" + 0.006*"사람" + 0.005*"벚꽃" + 0.005*"날씨" '
  '+ 0.005*"폭포" + 0.004*"가족" + 0.004*"요금" + 0.004*"물놀이"')]


In [13]:
top_topics = model.top_topics(corpus_LDA)
top_topics 

[([(0.008532017, '바다'),
   (0.007435218, '위치'),
   (0.0065295785, '하우스'),
   (0.005950785, '가족'),
   (0.00565107, '서귀포'),
   (0.005360794, '거리'),
   (0.0044677677, '시설'),
   (0.003944621, '마당'),
   (0.0033102708, '카페'),
   (0.0032529072, '애월'),
   (0.0032435188, '생활'),
   (0.0032173924, '감성'),
   (0.0031897286, '분위기'),
   (0.0031345459, '힐링'),
   (0.002929392, '오션'),
   (0.0029026314, '주변'),
   (0.0028906197, '테이블'),
   (0.0028083476, '커피'),
   (0.0027849872, '사람'),
   (0.002684658, '테라스')],
  -1.5370684564970298),
 ([(0.015260089, '오름'),
   (0.01474051, '코스'),
   (0.012015006, '바다'),
   (0.010910882, '올레길'),
   (0.008248668, '해수욕장'),
   (0.008175294, '사람'),
   (0.0076279705, '해변'),
   (0.00659035, '풍경'),
   (0.0056471177, '숲길'),
   (0.005269104, '한라산'),
   (0.005115993, '마을'),
   (0.004800289, '바람'),
   (0.004341899, '함덕'),
   (0.0043284358, '날씨'),
   (0.004043524, '캠프'),
   (0.0040001245, '나무'),
   (0.003727931, '정상'),
   (0.003587842, '사려'),
   (0.0035154573, '자연'),
   (0.003437492,

In [40]:
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -1.7964.
[([(0.01633619, '카페'),
   (0.0112707475, '바다'),
   (0.008001221, '사람'),
   (0.005564089, '맛집'),
   (0.0051417584, '서귀포'),
   (0.004922713, '구경'),
   (0.004701871, '날씨'),
   (0.004456012, '애월'),
   (0.0043847184, '커피'),
   (0.0040767547, '주차'),
   (0.00391286, '메뉴'),
   (0.0038187073, '가족'),
   (0.0036291163, '산책'),
   (0.0036137425, '폭포'),
   (0.0035743725, '체험'),
   (0.0035304755, '주차장'),
   (0.003271304, '바람'),
   (0.0032632903, '풍경'),
   (0.0031901973, '거리'),
   (0.0031835558, '해변')],
  -1.554515918452394),
 ([(0.015573673, '오름'),
   (0.014012412, '카페'),
   (0.008475053, '사람'),
   (0.008451729, '바다'),
   (0.00760581, '해수욕장'),
   (0.0058782655, '맛집'),
   (0.005856765, '날씨'),
   (0.005307213, '코스'),
   (0.0046243705, '김밥'),
   (0.0043938807, '커피'),
   (0.0041756905, '해변'),
   (0.004086145, '바람'),
   (0.0037284042, '나무'),
   (0.0037181377, '산책'),
   (0.0037178015, '남편'),
   (0.0037173233, '풍경'),
   (0.0037054014, '정상'),
   (0.0037052878, '식당'),
   (0.0